* Important Note: 

Backtesting strategy for EW and Sharpe optimization strategy has not been finished.

Only Mean Reverting Portfolio Optimizer (see main.py and visualization.ipynb) is finished.

# data loading and initial filtering

In [1]:
import math
from data_loader import Data_loader
from pair_filter import NPD_filter
from pair_filter import OLS_filter
import matplotlib.pyplot as plt



# loading data
start_idx = 0


dl = Data_loader(num_month_train=8, num_month_test=2)
window_train, window_test = dl.get_window(start_idx=start_idx)


# NPD selection
NPD = NPD_filter(window_train)
pairs, distance = NPD.select_pairs(percentile=5, verbose=True)

# OLS selection
OLS = OLS_filter(window_train)
pairs, params = OLS.select_pairs(pairs, verbose=True, confidence_level=0.01)


35432 pairs selected
min distance 0.0
max distance 0.7758473415183631
5938 out of 35432 pairs passed the test
pass rate: 0.17


# select top-k pairs accorindg to train sharpe ratio

In [2]:
from backtest import *

top_k = 50

bt = PairTradingBacktester_TrainTest(pairs[:300], params[:300], window_train, window_test) # for testing purpose
# bt = PairTradingBacktester_TrainTest(pairs, params, window_train, window_test) # complete
summary = bt.run(top_k=top_k)


spreads_train  = bt.train_spreads_df
spreads_test  = bt.test_spreads_df
returns_test   = bt.test_returns_df
returns_train   = bt.train_returns_df
comp_weights=bt.comp_weights

# EW optimizer

In [3]:
from optimizers import EW_optimizer

optimal_weights=EW_optimizer(returns_train) # train on training data

port_ret=(optimal_weights*returns_test).sum(axis=1) # test on testing data


port_ret.mean()*np.sqrt(252)/port_ret.std() # calculate out-of-sample sharpe ratio

(CVXPY) May 11 04:36:25 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/keyangpan/anaconda3/lib/python3.11/site-packages/cvxopt/base.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib\n  Referenced from: <E25E40AB-7857-39B9-8DE7-28B7B0E4806B> /Users/keyangpan/anaconda3/lib/python3.11/site-packages/cvxopt/base.cpython-311-darwin.so\n  Reason: tried: '/Users/keyangpan/anaconda3/lib/python3.11/site-packages/cvxopt/../../../liblapack.3.dylib' (no such file), '/Users/keyangpan/anaconda3/lib/python3.11/site-packages/cvxopt/../../../liblapack.3.dylib' (no such file), '/Users/keyangpan/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/keyangpan/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)")
(CVXPY) May 11 04:36:25 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlope

8.285872668905737

# Sharpe Optimizer

In [4]:
from optimizers import Sharpe_optimizer

optimal_weights=Sharpe_optimizer(returns_train)  # train on training data
port_ret=(optimal_weights*returns_test).sum(axis=1) # test on testing data
port_ret.mean()*np.sqrt(252)/port_ret.std() # calculate out-of-sample sharpe ratio

8.662738592720032